In [30]:
import tdmclient.notebook
await tdmclient.notebook.start()


NodeLockError: Node lock error (current status: busy)

In [2]:
import time
import sys
import cv2
import pygame as pg
import numpy as np
from importlib import reload

import motion_control
import vision
import kalman
import AlgoGlobNav as gn

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
reload(motion_control)
reload(vision)
reload(kalman)
reload(gn)

In [4]:
%reset


Nothing done.


In [23]:

vid = cv2.VideoCapture(1)


In [24]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target

    motor_left_target = l_speed
    motor_right_target = r_speed

In [25]:
@tdmclient.notebook.sync_var
def read_motors():
    """
    Read the motor speeds of the Thymio
    """
    global motor_left_speed, motor_right_speed

    return [motor_left_speed, motor_right_speed]


In [26]:
@tdmclient.notebook.sync_var
def read_proximity():
    """
    Read the proximity sensors of the Thymio
    """
    global prox_horizontal

    return prox_horizontal

In [31]:
%%run_python
nf_leds_prox_h(0,0,0,0,0,0,0,0)
nf_leds_prox_v(0,0)
nf_leds_rc(0)
nf_leds_sound(0)
nf_leds_temperature(0,0)
nf_leds_top(0,0,0)
nf_leds_bottom_left(0,0,0)
nf_leds_bottom_right(0,0,0)
nf_leds_buttons(30,0,0,0)
nf_leds_top(0,255,0)


In [34]:

# Kalman filter
t = time.time()
print("Start Kalman filter initialization")
mus_pos = []
mus_att = []

# Initialize state estimate using vision
try:
    pos, orientation = vision.fetchOdoMeters(vid, 1000)
except:
    print("Could not find initial state")

# Initialize kalman filters
try:
    kf_pos = kalman.Kalman(_acc_variance=0.5, _type="pose", init_x=np.hstack((pos,[0,0])))
    kf_att = kalman.Kalman(_acc_variance=0.5, _type="orientation", init_x=np.hstack((orientation,0)))
except:
    print("Could not initialize Kalman filters")

print("Initialization complete: %5.3f s" % (time.time()-t))
print("Initial pose:", kf_pos.mean)
print("Initial orientation", kf_att.mean*180/np.pi)
print("--------------------------------------")


# Global navigation
t = time.time()
print("Start global navigation initialization")

goal = vision.goalFetch(vid)
nodes, nodeCon, maskObsDilated, optimal_path = gn.opt_path(vid, goal)

print("Initialization complete: %5.3f s" % (time.time()-t))
print("--------------------------------------")


# Motion control
t = time.time()
print("Start motion control initialization")

mc = motion_control.MotionControl(optimal_path*100,
                                  kf_pos.mean[0]*100,
                                  kf_pos.mean[1]*100,
                                  kf_att.mean[0],
                                  nodes,
                                  nodeCon,
                                  maskObsDilated,
                                  goal)

print("Initialization complete: %5.3f s" % (time.time()-t))
print("--------------------------------------")


# Initialize pygame
pg.init()

h = int(841 / 2)
w = int(1189 / 2)
scr = pg.display.set_mode((w, h))

thymio = pg.image.load("thymio.png")
thymio = pg.transform.scale(thymio, (50, 50))

# main loop
running = True
simulate = False
local = False
while running:
    # Configure key presses for pygame and cv2 windows
    for event in pg.event.get():
        if event.type == pg.QUIT:
            cv2.destroyAllWindows()
            pg.quit()
            sys.exit()
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_RETURN :
                simulate = not simulate
    # Set up environment
    scr.fill((0, 0, 0))
    pg.draw.line(scr, (255, 255, 255), (0, int(h/2)), (w, int(h/2)))
    pg.draw.line(scr, (255, 255, 255), (int(w/2), 0), (int(w/2), h))

    # Start thymio movement
    if simulate:
        # Save expected values and covariance matrices
        mus_pos.append(kf_pos.mean)
        mus_att.append(kf_att.mean)

        # Read sensor data
        try:
            pos, orientation = vision.fetchOdoMeters(vid)
            fetched = True
        except:
            fetched = False
            print("No vision measurements found")

        motor_speeds = read_motors()
        speed = (sum(motor_speeds)/2)*0.0004                                # magnitude of velocity [m/s]
        angular_speed = (motor_speeds[1]-motor_speeds[0])*0.0004/0.094      # magnitude of angular velocity [rad/s]
        vel = [speed*np.cos(mus_att[-1][0]), speed*np.sin(mus_att[-1][0])]

        # Estimate pose
        kalman.estimate_pose(time.time()-t, kf_pos, vel, pos, fetched)

        # Estimate orientation
        kalman.estimate_orientation(time.time()-t, kf_att, angular_speed, orientation, fetched)

        # Motion control
        prox_front = read_proximity()
        mc.update_motion(kf_pos.mean[0]*100, kf_pos.mean[1]*100, kf_att.mean[0], prox_front, vid)

        # Update motor speeds
        motors(l_speed=int(mc.l_speed), r_speed=int(mc.r_speed))

        # Draw Thymio and optimal path in window
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        kalman.path(mc.optimal_path/100, w, h, scr)
    else:
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        kalman.path(mc.optimal_path/100, w, h, scr)
        motors(0, 0)

    # Update windows
    optimal_path = mc.optimal_path/100
    try:
        output = vision.liveFeedback(vid, mc.nodes, mc.nodeCon, mc.maskObsDilated, optimal_path)
    except:
        pass
    else:
        cv2.imshow('Live Feedback', output)

    pg.display.flip()
    t = time.time()

print("===================================")
print("-------------Finished--------------")
print("===================================")

Start Kalman filter initialization
Initialization complete: 0.010 s
Initial pose: [0.07533958 0.21200208 0.         0.        ]
Initial orientation [-88.75576241   0.        ]
--------------------------------------
Start global navigation initialization
Initialization complete: 0.119 s
--------------------------------------
Start motion control initialization
Initialization complete: 0.000 s
--------------------------------------
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measurements found
No vision measu

SystemExit: 

In [ ]:
motors(100, 100)
stop()

In [ ]:

motors(0,0)

In [21]:
stop()

In [ ]:
\\